## 搭建一个分词工具

### Part 1  基于枚举方法来搭建中文分词工具

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率 word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 对于给定字符串：”我们学习人工智能，人工智能是未来“, 找出所有可能的分割方式
- [我们，学习，人工智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工，智能，是，未来]
- [我们，学习，人工智能，人工，智能，是，未来]
.......


#### Step 2: 我们也可以计算出每一个切分之后句子的概率
- p(我们，学习，人工智能，人工智能，是，未来)= -log p(我们)-log p(学习)-log p(人工智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工)-log p(智能)-log p(是)-log p(未来)
- p(我们，学习，人工智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工智能)-log p(人工)-log p(智能)-log(是)-log p(未来)
.....

#### Step 3: 返回第二步中概率最大的结果

In [1]:
# TODO: 第一步： 从综合类中文词库.xlsx 中读取所有中文词。
#  hint: 思考一下用什么数据结构来存储这个词典会比较好？ 要考虑我们每次查询一个单词的效率。 

# 以下是每一个单词出现的概率。为了问题的简化，我们只列出了一小部分单词的概率。 在这里没有出现的的单词但是出现在词典里的，统一把概率设置成为0.00001
# 比如 p("学院")=p("概率")=...0.00001

import pandas as pd
import numpy as np

vocab = {term:1e-5 for term in pd.read_excel("worddict.xlsx", header=None).drop(columns=[1, 2]).iloc[:, 0]}

word_prob = {"北京":0.03,"的":0.08,"天":0.005,"气":0.005,"天气":0.06,"真":0.04,"好":0.05,"真好":0.04,"啊":0.01,"真好啊":0.02, 
             "今":0.01,"今天":0.07,"课程":0.06,"内容":0.06,"有":0.05,"很":0.03,"很有":0.04,"意思":0.06,"有意思":0.005,"课":0.01,
             "程":0.005,"经常":0.08,"意见":0.08,"意":0.01,"见":0.005,"有意见":0.02,"分歧":0.04,"分":0.02, "歧":0.005}

print(sum(word_prob.values()))
vocab.update(word_prob)

MAX_WORD_LEN = np.max([len(term) for term in vocab])
print("Max word lenth is {}".format(MAX_WORD_LEN))

1.0000000000000002
Max word lenth is 16


In [2]:
def segment(text):
    """
    递归算法
    """
    if len(text) == 0:
        return [[]]
    
    str_len = len(text)
    seg_plans = []    

    # 考虑切割的end索引可以取到的值, 注意slice区间为[start_idx, end_idx)，
    # 故end_idx的取值范围为 [1, str_len/MAX_WORD_LEN]，等价于 [1, str_len/MAX_WORD_LEN + 1)
    end_idx_upper = min(str_len, MAX_WORD_LEN) + 1
    
    # 遍历end_idx
    for end_idx in range(1, end_idx_upper):
        span = text[:end_idx]
        
        # 如果span是词汇或单字，把text - span后的剩下部分递归处理
        # 将递归返回的每一种后续切割方案加上span后返回
        if span in vocab or len(span) == 1:
            sub_seg_plans = segment(text[end_idx:])
            for a_plan in sub_seg_plans:
                seg_plans.append([span] + a_plan)
                
    return seg_plans

In [3]:
segment("我爱北京天安门")

[['我', '爱', '北', '京', '天', '安', '门'],
 ['我', '爱', '北', '京', '天', '安门'],
 ['我', '爱', '北', '京', '天安', '门'],
 ['我', '爱', '北', '京', '天安门'],
 ['我', '爱', '北京', '天', '安', '门'],
 ['我', '爱', '北京', '天', '安门'],
 ['我', '爱', '北京', '天安', '门'],
 ['我', '爱', '北京', '天安门'],
 ['我爱', '北', '京', '天', '安', '门'],
 ['我爱', '北', '京', '天', '安门'],
 ['我爱', '北', '京', '天安', '门'],
 ['我爱', '北', '京', '天安门'],
 ['我爱', '北京', '天', '安', '门'],
 ['我爱', '北京', '天', '安门'],
 ['我爱', '北京', '天安', '门'],
 ['我爱', '北京', '天安门']]

In [4]:
def neg_loglikelihood(seg_plan):
    return np.sum([-np.log(vocab[term]) for term in seg_plan])

seg_plans = segment("我爱北京天安门")
nlls = list(map(neg_loglikelihood, seg_plans))

sorted(zip(nlls, seg_plans), key=lambda entry: entry[0])

[(26.53240882726044, ['我爱', '北京', '天安门']),
 (31.830726193808474, ['我爱', '北京', '天', '安门']),
 (38.04533429223067, ['我', '爱', '北京', '天安门']),
 (38.04533429223067, ['我爱', '北京', '天安', '门']),
 (43.343651658778704, ['我', '爱', '北京', '天', '安门']),
 (43.343651658778704, ['我爱', '北京', '天', '安', '门']),
 (46.051701859880914, ['我爱', '北', '京', '天安门']),
 (49.5582597572009, ['我', '爱', '北京', '天安', '门']),
 (51.35001922642895, ['我爱', '北', '京', '天', '安门']),
 (54.856577123748934, ['我', '爱', '北京', '天', '安', '门']),
 (57.564627324851145, ['我', '爱', '北', '京', '天安门']),
 (57.564627324851145, ['我爱', '北', '京', '天安', '门']),
 (62.86294469139918, ['我', '爱', '北', '京', '天', '安门']),
 (62.86294469139918, ['我爱', '北', '京', '天', '安', '门']),
 (69.07755278982137, ['我', '爱', '北', '京', '天安', '门']),
 (74.3758701563694, ['我', '爱', '北', '京', '天', '安', '门'])]

### Part 2  基于维特比算法来优化上述流程

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 根据词典，输入的句子和 word_prob来创建带权重的有向图（Directed Graph） 参考：课程内容
有向图的每一条边是一个单词的概率（只要存在于词典里的都可以作为一个合法的单词），这些概率已经给出（存放在word_prob）。
注意：思考用什么方式来存储这种有向图比较合适？ 不一定只有一种方式来存储这种结构。 

#### Step 2: 编写维特比算法（viterebi）算法来找出其中最好的PATH， 也就是最好的句子切分
具体算法参考课程中讲过的内容

#### Step 3: 返回结果
跟PART 1的要求一致

In [5]:
SINGLE_TERM_WORD_SCORE = 1e-5
def constructe_graph(text, vocab):
    """
    构建词图
    """
    s_len = len(text)
    graph = [[] for i in range(s_len)]
    
    for start_idx in range(s_len):
        for end_idx in range(start_idx, s_len):
            span = text[start_idx:end_idx + 1]
            # 如果Span是单词，加上edge
            # edge连接到span之前，也就是上个有效单词的结尾（对于句首单词，为-1）
            if span in vocab:
                graph[end_idx].append((start_idx - 1, -np.log(vocab[span])))
            
            # 加入缺省的单字edge
            elif start_idx == end_idx:
                graph[end_idx].append((start_idx - 1, -np.log(SINGLE_TERM_WORD_SCORE)))
    return graph
# test code
print(constructe_graph("经", vocab))

[[(-1, 11.512925464970229)]]


In [6]:
## TODO 请编写word_segment_viterbi函数来实现对输入字符串的分词
def word_segment_viterbi(input_str):
    """
    1. 基于输入字符串，词典，以及给定的unigram概率来创建DAG(有向图）。
    2. 编写维特比算法来寻找最优的PATH
    3. 返回分词结果
    
    input_str: 输入字符串   输入格式：“今天天气好”
    best_segment: 最好的分词结果  输出格式：["今天"，"天气"，"好"]
    """
    s_len = len(input_str) 
    if s_len <= 1:
        return [input_str]    
    # TODO: 第一步：根据词典，输入的句子，以及给定的unigram概率来创建带权重的有向图（Directed Graph） 参考：课程内容
    #      有向图的每一条边是一个单词的概率（只要存在于词典里的都可以作为一个合法的单词），这些概率在 word_prob，如果不在word_prob里的单词但在
    #      词典里存在的，统一用概率值0.00001。
    #      注意：思考用什么方式来存储这种有向图比较合适？ 不一定有只有一种方式来存储这种结构。 
 
    graph = constructe_graph(input_str, vocab)
    # TODO： 第二步： 利用维特比算法来找出最好的PATH， 这个PATH是P(sentence)最大或者 -log P(sentence)最小的PATH。
    #              hint: 思考为什么不用相乘: p(w1)p(w2)...而是使用negative log sum:  -log(w1)-log(w2)-...
    # 初始化动态规划数组
    path = [-1] * s_len
    dp   = [ 0] * s_len
    
    for i in range(s_len):
        # 自底向上计算每个dp位置
        dp[i] = 10e20
        for edge in graph[i]:
            # 计算neg log prob分数
            # 记录最大的路径
            prev_score = dp[edge[0]] if edge[0] >= 0 else 0
            score = prev_score + edge[1]
            if score < dp[i]:
                dp[i] = score
                path[i] = edge[0]

    # TODO: 第三步： 根据最好的PATH, 返回最好的切分
    idx = s_len - 1
    best_segment = []
    while idx > -1:
        best_segment.append(input_str[path[idx] + 1:idx + 1])
        idx = path[idx]

    best_segment.reverse()
    return best_segment      

In [7]:
# 测试
print(word_segment_viterbi("北京的天气真好啊"))
print(word_segment_viterbi("今天的课程内容很有意思"))
print(word_segment_viterbi("经常有意见分歧"))
print(word_segment_viterbi("经馨"))
print(word_segment_viterbi(""))

['北京', '的', '天气', '真好啊']
['今天', '的', '课程', '内容', '很有', '意思']
['经常', '有意见', '分歧']
['经', '馨']
['']



# TODO: 第一种方法和第二种方法的时间复杂度和空间复杂度分别是多少？
第一个方法： 
时间复杂度= $O(2^n)$ , 空间复杂度= $O(2^n)$

第二个方法：
时间复杂度= $O(n^2)$ , 空间复杂度= $O(n^2)$

# TODO：如果把上述的分词工具持续优化，有哪些可以考虑的方法？ （至少列出3点）
- 0. （例）， 目前的概率是不完整的，可以考虑大量的语料库，然后从中计算出每一个词出现的概率，这样更加真实
- 1. 目前使用的语言模型仅考虑了unigram， 考虑加入bigram，trigram等语言模型的条件概率
- 2. 对于未出现的词汇同意考虑了默认30的score值，这和实际情况不符合考虑加入add-one smoothing ，interpolation，或者good-turing smooth
- 3. 如果输入句子中有错误的时候是否考虑加入拼写纠错功能？
- 4. 可以看到算法的复杂度是输入文本的长度的二次方，如果输入一篇文章的时候，复杂度会过高。这需要进行优化,比如先讲输入的文档段句（可以根据句号，逗号等使用规则进行段句，接着再进行分词。这样能降低复杂度。